# Interpolation

As mentioned in [elements.ipynb](elements.ipynb) `interpolate()` cannot work in Hermite function spaces without extending the interface for `ufc::function` to include differentiation. This comes from the fact that the standard (local) nodal interpolant is given by

$$ I_K (f) = \sum_{i = 1}^{n_K} L_i (f) \phi_i $$

for all dofs $L_i$ over cell $K$. In particular we need to let the partial differentiation along each spatial coordinate (in the notation of the paper $L_{\alpha_i}, 1 \leq i \leq d$) act on the function $f$ and use this value as the coefficient for the corresponding shape function $\phi_{\alpha_i}$ in the representation in the global basis of $V_h$.

This notebook will contain progress in this matter.

In [2]:
from dolfin import *

mesh = UnitSquareMesh(10,10)
V = FunctionSpace(mesh, 'Hermite', 3)
f = Expression("x[0]*x[1]", degree=3)
v = interpolate(f, V)
_ = plot(v, cmap='bone')

RuntimeError: evaluate_dof(s) for Hermite element not implemented.

## A test

The first thing to check once `evaluate_dof()` has been implemented for Hermite elements is a constant:

In [ ]:
u = interpolate(Constant(2.0), V)
_ = plot(u, cmap='bone')